https://www.investopedia.com/terms/c/cumulativereturn.asp

https://mathworld.wolfram.com/CumulativeProduct.html

minimum amount: https://stackoverflow.com/questions/61582902/python-binance-api-apierrorcode-1013-filter-failure-lot-size

In [1]:
import sqlalchemy
import pandas as pd
from binance.client import Client

api_key = 'Your key'
api_secret = 'Your secret'
client = Client(api_key,api_secret)

engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

In [2]:
df = pd.read_sql('BTCUSDT', engine) #we only process 6 data points back then, but we could keep it moving
df

,Symbol,Time,Price
0,BTCUSDT,2021-06-16 21:56:08.515,38786.13
1,BTCUSDT,2021-06-16 21:56:08.520,38785.18
2,BTCUSDT,2021-06-16 21:56:08.604,38786.13
3,BTCUSDT,2021-06-16 21:56:08.847,38786.13
4,BTCUSDT,2021-06-16 21:56:08.890,38786.13
5,BTCUSDT,2021-06-16 21:56:08.890,38786.13
6,BTCUSDT,2021-06-16 21:56:09.249,38786.12
7,BTCUSDT,2021-06-16 21:56:09.249,38786.12
8,BTCUSDT,2021-06-16 21:56:09.550,38786.12
9,BTCUSDT,2021-06-16 21:56:09.619,38786.12


In [3]:
#Strategy: Trend following 
#if the crypto was rising by x% -> buy
#exit when profit is above 0.15% or loss is crossing -0.15%

In [4]:
def strategy(entry, lookback, qty, open_position=False):
#entry: x mention above, lookback: how much time do we want to go back, quantity we want to trade, boolean
    while True:
        df = pd.read_sql('BTCUSDT', engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change()+1).cumprod()-1 #cumulative return #cumulative product
        if not open_position: #if we don't have a open position
            if cumret[cumret.last_valid_index()] > entry:#most recent cumulative return
                order = client.create_order(symbol='BTCUSDT',side='BUY',type='MARKET',quantity=qty)#place b order
                print(order)
                cost = float(order['fills'][0]['price'])*qty
                commission1 = float(order['fills'][0]['commission'])
                print(f'Cost of transactions: ${cost}USDT')
                open_position = True #break the while loop
                break
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.Time>pd.to_datetime(order['transactTime'],unit='ms')]
            #slicing for timestamp after transaction
            
            if len(sincebuy)>1: #becuz we are using second updated data we could have got empty sincebuy
                sincebuyret = (sincebuy.Price.pct_change()+1).cumprod()-1
                last_entry = sincebuyret[sincebuyret.last_valid_index()] #last entry: cumulative return untill now
                if last_entry>0.0015 or last_entry<-0.0015:
                    if qty-commission1>0.001:
                        amount = qty-commission1
                    else:
                        amount = 0.001 #minimum amount
                    order = client.create_order(symbol='BTCUSDT',side='SELL',type='MARKET',quantity=float(amount))
                    print(order)
                    if len(order)==14:
                        profit=0
                        for n in range(len(order["fills"])):
                            profit += (float(order["fills"][n]["qty"])*float(order["fills"][n]["price"]))-float(order["fills"][n]["commission"])
                        profit -= cost
                        print(f'gain/loss for this trade: ${profit}USTD')
                    break
                    

In [5]:
strategy(0.001,60,0.001)

{'symbol': 'BTCUSDT', 'orderId': 6484725293, 'orderListId': -1, 'clientOrderId': 'Xt6exsAFVGCNQYoz2cYrPI', 'transactTime': 1623880745999, 'price': '0.00000000', 'origQty': '0.00100000', 'executedQty': '0.00100000', 'cummulativeQuoteQty': '38.75404000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '38754.04000000', 'qty': '0.00100000', 'commission': '0.00000100', 'commissionAsset': 'BTC', 'tradeId': 914147214}]}
Cost of transactions: $38.75404USDT
{'symbol': 'BTCUSDT', 'orderId': 6484760282, 'orderListId': -1, 'clientOrderId': 'FsKnrSNasZc3q2AkQrrvDm', 'transactTime': 1623881019095, 'price': '0.00000000', 'origQty': '0.00100000', 'executedQty': '0.00100000', 'cummulativeQuoteQty': '38.81539000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '38815.39000000', 'qty': '0.00100000', 'commission': '0.03881539', 'commissionAsset': 'USDT', 'tradeId': 914150472}]}
gain/loss for this trade: $0.022